# Pre-processing of polychromatic beam data

In this notebook, the polychromatic beam X-ray radiographs are converted into projected density maps and prepared for reconstruction.

---

## Environment setup

In [1]:
import h5py
import ipywidgets as widgets
import numpy as np

from ipywidgets import (
                        fixed,
                        interact
                        )
from matplotlib import pyplot as plt

from preproc_utilities import (
                               getDataPB,
                               convertPB,
                               showSprayPB
                               )


# Data directories
inp_fld = 'inputs'
out_fld = 'outputs'

### Version information

In [2]:
%reload_ext watermark
%watermark -p h5py,ipywidgets,numpy,matplotlib,cv2,scipy,skimage

h5py      : 3.6.0
ipywidgets: 7.6.5
numpy     : 1.17.0
matplotlib: 3.4.3
cv2       : 4.5.1
scipy     : 1.6.2
skimage   : 0.18.1



## Camera calibrations
Load in the calculated camera calibration parameters (see: [pb01_calibration.ipynb](./pb01_calibration.ipynb)).

In [3]:
cal_c1 = np.load(f'{out_fld}/cam1_calibration.npz', allow_pickle=True)
cal_c2 = np.load(f'{out_fld}/cam2_calibration.npz', allow_pickle=True)
cal_c3 = np.load(f'{out_fld}/cam3_calibration.npz', allow_pickle=True)
camCal = [x['cal'].tolist() for x in [cal_c1, cal_c2, cal_c3]]

## Raw spray data

The raw images for both $Re$ cases (open rim condition at $Re$ = 7,100; impact wave condition at $Re$ = 10,600) are located in the `<inputs/*frame0.hdf5>` files. For the purposes of showing quick reproducibility of the code, only the first frame of the data sets are shown here in this repository. The rest of the dataset can be found over at [[1]](#References).

For data compression, some light pre-processing was done on the supplied raw images found in the HDF5 files. The images were initially captured on FASTCAM SA-Z at 20 kHz on a 1024$\times$1024 px frame size as 12-bit TIFFs. They have been compressed down to a 512$\times$512 px window and averaged down to an effective 10 kHz time sequence for data compression. Temporal averages of all collected 10 kHz images are also given. The final volumes shown in the corresponding publication are calculated using these compressed data sets.

Let's load in the first frames and averaged frame for each case.

In [4]:
# PB: Polychromatic beam
# lo: low Reynolds number (7100), hi: high Reynolds number (10600)
pbFile_lo = f'{inp_fld}/spray_polychromatic_Re-7100_frame0.hdf5'
pbFile_hi = f'{inp_fld}/spray_polychromatic_Re-10600_frame0.hdf5'

# Time averaged case
pbFileAvg_lo = f'{inp_fld}/spray_polychromatic_Re-7100_averaged.hdf5'
pbFileAvg_hi = f'{inp_fld}/spray_polychromatic_Re-10600_averaged.hdf5'

# Swapping axes to be consistent with the other data shapes
pbRaw_lo = getDataPB(pbFile_lo, '/').swapaxes(0, 1)
pbRaw_hi = getDataPB(pbFile_hi, '/').swapaxes(0, 1)
pbRawAvg_lo = getDataPB(pbFileAvg_lo, '/').swapaxes(0, 1)
pbRawAvg_hi = getDataPB(pbFileAvg_hi, '/').swapaxes(0, 1)

The raw images were captured as TIFFs with 12-bit pixel intensities (shown here as arbitrary units).

In [5]:
interact(
         showSprayPB, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         pb=fixed((pbRaw_lo, pbRaw_hi)),
         Re=fixed((7100, 10600)),
         units=fixed('a.u.'),
         cmap=widgets.Dropdown(options=plt.colormaps(),
                               description='Colormap', 
                               value='gray')
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Dro…

In [6]:
interact(
         showSprayPB, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         pb=fixed((pbRawAvg_lo, pbRawAvg_hi)),
         Re=fixed((7100, 10600)),
         units=fixed('a.u.'),
         cmap=widgets.Dropdown(options=plt.colormaps(),
                               description='Colormap', 
                               value='gray')
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Dro…

## Preparing spray images for reconstruction
The bulk of the spray processing can be found in the `convertPB()` function (see: [utilities.py](./utilities.py)). This function handles conversion to normalized transmission units, masking off of background, calculation of equivalent path length (EPL), and final conversion to projected density maps.

In [7]:
# Otsu scale controls the thresholding strength for the background masking
# routine--see the convertPB() function for detail.
otsuScale_lo = [0.8, 0.8, 0.8]
pbDens_lo = convertPB(pbFile_lo, 'Re-7100', otsuScale_lo, camCal)
pbAvg_lo = convertPB(pbFileAvg_lo, 'Re-7100', otsuScale_lo, camCal)

otsuScale_hi = [0.6, 0.9, 0.5]
pbDens_hi = convertPB(pbFile_hi, 'Re-10600', otsuScale_hi, camCal)
pbAvg_hi = convertPB(pbFileAvg_hi, 'Re-10600', otsuScale_hi, camCal)

Let's visualize the final prepared images, now with units of projected density (μg/mm$^2$).

In [8]:
interact(
         showSprayPB, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         pb=fixed((pbDens_lo, pbDens_hi)),
         Re=fixed((7100, 10600)),
         units=fixed('μg/mm$^2$'),
         cmap=widgets.Dropdown(options=plt.colormaps(),
                               description='Colormap', 
                               value='Blues')
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Dro…

In [9]:
interact(
         showSprayPB, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         pb=fixed((pbAvg_lo, pbAvg_hi)),
         Re=fixed((7100, 10600)),
         units=fixed('μg/mm$^2$'),
         cmap=widgets.Dropdown(options=plt.colormaps(),
                               description='Colormap', 
                               value='Blues')
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Dro…

## Save dataset
The processed images are saved as HDF5 files for further processing.

In [10]:
with h5py.File(f'{out_fld}/spray_pbDens_frame0.hdf5', 'w') as hf:
    hf.create_dataset('Re-7100', data=pbDens_lo, compression='gzip')
    hf.create_dataset('Re-10600', data=pbDens_hi, compression='gzip')
    
with h5py.File(f'{out_fld}/spray_pbDens_averaged.hdf5', 'w') as hf:
    hf.create_dataset('Re-7100', data=pbAvg_lo, compression='gzip')
    hf.create_dataset('Re-10600', data=pbAvg_hi, compression='gzip')

## Next steps
After processing of the raw spray images, the next step in the workflow is [pb03_refinement.ipynb](./pb03_refinement.ipynb).

## References

1. Rahman, N. X-ray radiography datasets for tomographic reconstruction of liquid jet breakup dynamics. *Purdue University Research Repository* (2022). doi: [10.4231/G20X-4Z27](https://doi.org/10.4231/G20X-4Z27).

2. Halls, B. R. et al. Quantitative 10-50 kHz X-ray radiography of liquid spray distributions using a rotating-anode tube source. *Int. J. Multiphas. Flow* 109, 123–130 (2018). doi: [10.1016/j.ijmultiphaseflow.2018.07.014](https://doi.org/10.1016/j.ijmultiphaseflow.2018.07.014).

---

**Author**: Naveed Rahman, Purdue University, 23 March 2022

---